# Restrained positions in OpenMM

This notebook shows how to add soft (harmonic) restraints to the absolute positions of some atoms -- in this case, the backbone carbons.

Much of this setup is based on https://github.com/ParmEd/ParmEd/wiki/OpenMM-Tricks-and-Recipes#positional-restraints

In [1]:
import openmm_scaled_md as scaled
import simtk.openmm as mm
from simtk.openmm import app
from simtk import unit

import mdtraj as md

In [2]:
pdb = app.PDBFile('../resources/AD_initial_frame.pdb')
forcefield = app.ForceField('amber96.xml', 'tip3p.xml')

pdb_system = forcefield.createSystem(pdb.topology,
                                     nonbondedMethod=app.PME, 
                                     nonbondedCutoff=1.0*unit.nanometers,
                                     constraints=app.HBonds,
                                     rigidWater=True,
                                     ewaldErrorTolerance=0.0005)

The first two cells are identical to the normal setup.

The next cells add restraints on the absolute positions. We'll use [MDTraj's atom selection language](http://mdtraj.org/latest/atom_selection.html) to select atoms; this *does* assume that MDTraj does not change the atom ordering scheme when creating an `mdtraj.Topology` from an `openmm.app.Topology`. We'll also assume that we want to restrain to the absolute positions as given in the PDB.

In [3]:
# we use MDTraj to intelligently select the atoms we want (carbon backbone)
topology = md.Topology.from_openmm(pdb.topology)
pos_restrained_atoms = topology.select("element == 'C' and backbone")

In [4]:
# this is the stuff specific to positional restraints
force_constant = 5.0
force_unit = unit.kilocalories_per_mole / unit.angstroms**2
force = mm.CustomExternalForce("k*((x-x0)^2+(y-y0)^2+(z-z0)^2)")
force.addGlobalParameter("k", force_constant*force_unit)
force.addPerParticleParameter("x0")
force.addPerParticleParameter("y0")
force.addPerParticleParameter("z0")
for a_idx in pos_restrained_atoms:
    init_pos = pdb.positions.value_in_unit(unit.nanometer)[a_idx]
    force.addParticle(a_idx, init_pos)
pdb_system.addForce(force)
# returns the index of the force

5

### Running with restrained absolute positions

Now we'll create a trajectory that uses this system for dynamics. Getting the `simulation` object and using it to run MD is the same as is normally done.

In [5]:
# this is equivalent to the standard BAOAB integrator, since force_scaling is 1
integrator = scaled.integrators.BAOABIntegrator(
    temperature=300.0*unit.kelvin,
    collision_rate=1.0/unit.picosecond,
    timestep=2.0*unit.femtosecond,
    force_scaling=1.0
)

In [6]:
sim = app.Simulation(pdb.topology, pdb_system, integrator)
sim.context.setPositions(pdb.positions)
sim.reporters.append(app.PDBReporter('position_restrained.pdb', 10))

In [7]:
sim.step(1000)

### Visualizing with NGLView

If you also have [NGLView](http://nglviewer.org/nglview/latest/) installed, you can visualize the trajectory in-notebook.

In [8]:
import nglview as nv
traj = md.load("./position_restrained.pdb")

In [9]:
view = nv.show_mdtraj(traj)
view.add_ball_and_stick("ACE ALA NME")
view.add_point("water and .O")
view

NGLWidget(count=100)